In [13]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity


In[311]:

In [14]:

# Load the dataset
df = pd.read_csv('your_car_dataset.csv')



# Load dataset
path = r"C:\Users\RaviShankar\Downloads\expanded_restaurant_data.csv"  # Replace with the actual path
df = pd.read_csv(path)
df.head()

,Date,Restaurant Name,City,Sales (K),Inventory Cost (K),Quantity,Average Serve Time,Number of Customers,Cuisine Type,Male,Female,Likelihood_of_Return,Avg_Serve_Time_Rating,Avg_Ambience_Rating,Avg_Taste_of_Food_Rating,Avg_Hygiene_Rating,Overall_Rating
0,1/1/2023 13:34,The Daily Fix Nutrition,Kewanee,61.32,49.91,192,19.15,60,Healthy,40,20,3,2,2,2,3,2.4
1,1/2/2023 8:02,A Better Burger,Jonesville,196.77,38.65,85,11.30,50,American,30,20,5,1,4,4,4,3.6
2,1/3/2023 17:47,Casa Maya Grill,Deerfield Beach,163.26,46.50,410,17.31,50,Mexican,20,30,5,3,3,4,1,3.2
3,1/4/2023 0:36,Fitlife Nutricion,Chicago,186.74,30.11,287,16.26,50,American,30,20,4,1,3,3,4,3.0
4,1/5/2023 0:19,Saladworks,Bethlehem,193.35,42.20,166,19.59,90,American,50,40,2,3,3,2,1,2.2


In[312]:

In [15]:


df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      69 non-null     object 
 1   Restaurant Name           69 non-null     object 
 2   City                      69 non-null     object 
 3   Sales (K)                 69 non-null     float64
 4   Inventory Cost (K)        69 non-null     float64
 5   Quantity                  69 non-null     int64  
 6   Average Serve Time        69 non-null     float64
 7   Number of Customers       69 non-null     int64  
 8   Cuisine Type              69 non-null     object 
 9   Male                      69 non-null     int64  
 10  Female                    69 non-null     int64  
 11  Likelihood_of_Return      69 non-null     int64  
 12  Avg_Serve_Time_Rating     69 non-null     int64  
 13  Avg_Ambience_Rating       69 non-null     int64  
 14  Avg_Taste_of

**Clean Up Columns**

In[313]:

Check the column names

In [16]:
print(df.columns)


Index(['Date', 'Restaurant Name', 'City', 'Sales (K)', 'Inventory Cost (K)',
       'Quantity', 'Average Serve Time', 'Number of Customers', 'Cuisine Type',
       'Male', 'Female', 'Likelihood_of_Return', 'Avg_Serve_Time_Rating',
       'Avg_Ambience_Rating', 'Avg_Taste_of_Food_Rating', 'Avg_Hygiene_Rating',
       'Overall_Rating'],
      dtype='object')


In [17]:
# Add an ID column if the dataset doesn't have one
df['Car_Id'] = range(1, len(df) + 1)

In[314]:

In [18]:
# Features to consider for recommendation
features = ['Age of Car', 'Distance', 'Owner', 'Ratings', 'Safety Rating']


In [19]:
# Normalize the data
scaler = MinMaxScaler()
df_scaled = df.copy()
df_scaled[features] = scaler.fit_transform(df[features])


In [20]:
# Invert the "lower is better" features
for col in ['Age of Car', 'Distance', 'Owner']:
    df_scaled[col] = 1 - df_scaled[col]


In [21]:
# Calculate similarity using cosine similarity
similarity_matrix = cosine_similarity(df_scaled[features])


In [22]:
# Convert similarity matrix to DataFrame
similarity_df = pd.DataFrame(similarity_matrix, index=df['Car_Name'], columns=df['Car_Name'])


In [23]:
# Create the car similarity data
car_similarity_data = []

In [24]:
# Iterate through each car and its similarity scores
for car_name in df['Car_Name']:
    # Get the index of the car
    idx = df.index[df['Car_Name'] == car_name].tolist()[0]
    car_id = df.loc[idx, 'Car_Id']
    

In [25]:
    # Get similarity scores of all cars with the current car
sim_scores = list(enumerate(similarity_matrix[idx]))


Restaurants_similarity.csv has been generated!


In[316]:

In [26]:
   # Sort cars based on similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    

Extract only numeric values from 'Runtime'

In [27]:
    # Extract the car names, IDs, and similarity scores (skip the first one since it's the car itself)
    for i, score in sim_scores[1:11]:
        similar_car_name = df.iloc[i]['Car_Name']
        similar_car_id = df.iloc[i]['Car_Id']
        car_similarity_data.append((car_name, car_id, similar_car_name, similar_car_id, score))


In[317]:

In [28]:
# Create a DataFrame for car similarities
car_similarity_df = pd.DataFrame(
    car_similarity_data, 
    columns=["Car_Name", "Car_Id", "Similar_Car", "Similar_Car_Id", "Cosine_Similarity"]
)

        Restaurant Name Cuisine Type  Likelihood_of_Return  \
44  Coastal Breeze Cafe     American                  0.00   
67      Bluebonnet Cafe        Latin                  0.00   
29        Casa De Sazon        Latin                  1.00   
66       Mesquite Table        Latin                  0.25   
49    Cuban Spice Grill        Latin                  0.25   

    Avg_Serve_Time_Rating  Avg_Ambience_Rating  Avg_Taste_of_Food_Rating  \
44                   1.00                 0.25                      0.00   
67                   0.25                 0.75                      0.50   
29                   0.50                 0.00                      0.75   
66                   0.00                 0.50                      0.00   
49                   0.00                 0.50                      0.50   

    Avg_Hygiene_Rating  Overall_Rating  Similarity Score  
44                0.75        0.333333          0.301932  
67                0.00        0.200000          0.17

In [ ]:
# Save the similarity data to a CSV file
car_similarity_df.to_csv('car_similarity.csv', index=False)
